#목적형 R&R 경안천 유역 대상 적용을 위한 예측 파일

In [1]:
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import  train_test_split, KFold
from sklearn.metrics import mean_absolute_error
from sklearn.utils import shuffle
from tqdm import tqdm
import time
import glob
import tensorflow as tf
import cv2
import gc
from PIL import Image
from keras.models import load_model
import datetime
os.environ["TF_ENABLE_ONEDNN_OPTS"] = '0'

2023-08-06 17:00:44.152130: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
import keras 
import keras.backend as K

from keras.layers import *
from keras.models import Model
from keras.losses import mean_absolute_error
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Nadam
from keras.metrics import AUC

print(f"Keras Version : {keras.__version__}")

def mae255(y_true, y_pred):
    return 255 * mean_absolute_error(y_true, y_pred)

Keras Version : 2.9.0


In [3]:
from keras import backend as K
import warnings
warnings.simplefilter("ignore")
# Metrics
# f1 score 계산
def Rain(array):
    R=(array*255.)
    return R

def CSI_m(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    y_true = y_true.reshape(1, -1)[0]
    y_pred = y_pred.reshape(1, -1)[0]
    remove_NAs = y_true > 0
    y_true = np.where(y_true[remove_NAs] >= 0.1, 1, 0)
    y_pred = np.where(y_pred[remove_NAs] >= 0.1, 1, 0)
    right=np.sum(y_true * y_pred == 1)
    #print(right,np.sum(y_pred),np.sum(y_true),right)
    CSI = right/(np.sum(y_pred)+np.sum(y_true)-right+1e-07)
    return CSI

def CSI_custom(y_true, y_pred):
    score = tf.py_function(func=CSI_m, inp=[y_true, y_pred], Tout=tf.float32,  name='CSI_custom') 
    return score

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred): 
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred): 
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def mae_custom(y_true, y_pred):
    y_pred=tf.convert_to_tensor(y_pred)
    y_true=tf.convert_to_tensor(y_true)
    y_true = Rain(y_true)
    y_pred = Rain(y_pred)  
    thr1 = K.greater(y_true,0.1)
    loss1 =K.mean(K.abs(y_true[thr1] - y_pred[thr1]))
    arr=[loss1]    
    value_not_nan = tf.dtypes.cast(tf.math.logical_not(tf.math.is_nan(arr)), dtype=tf.float32)
    loss0=tf.math.multiply_no_nan(arr, value_not_nan)
    loss=tf.convert_to_tensor(K.sum(loss0))
    return loss


#특정시점 예측해서 이미지 그리기

In [4]:
date_nm='202208082200'
print(date_nm[2:12])

2208082200


In [5]:
## data_loader for QPF
# READ HFC_COMP_COCM (PAST 4 data)
def read_ASC(timestamp):
#    print(timestams)
    YY = timestamp[0:2]
    MM = timestamp[2:4]
    DD = timestamp[4:6]
    HH = timestamp[6:8]
    MN = timestamp[8:10]
    RDR_time = timestamp[0:10]
	#RAD200621192000 = 20200621192000
    YYYY = '20'+YY
    print(RDR_time)
    #path='/data/lapisis_data/RDR_COMP/INPUT_ASC'
    path='/data'
    filename = path+'/RDR_CMP_HSP_PUB_20'+RDR_time+'-525x625-1km.asc'
    data = np.zeros([625,525], np.float32) #ASCII랑배열 반대
    data = np.loadtxt(filename,skiprows=6)
    data[data < 0] = 0.
    #data_re=data[46:302,186:442]*6. #256x256
    attr = datetime.datetime(int(YYYY),int(MM),int(DD),int(HH),int(MN))
    return data, attr

def qpe_data(filetime):
    YY = filetime[0:2]
    MM = filetime[2:4]
    DD = filetime[4:6]
    HH = filetime[6:8]
    MN = filetime[8:10]
    YYYY = '20'+YY
    latest_datetime = datetime.datetime(int(YYYY),int(MM),int(DD),int(HH),int(MN))
    #print(latest_datetime)
    list_for_qpf = [ts.strftime("%y%m%d%H%M") for ts in 
    [latest_datetime - datetime.timedelta(minutes=t) for t in [30, 20, 10, 0]]]
    input_scans = np.array([read_ASC(ts)[0] for ts in list_for_qpf])
    qpe_scans = np.concatenate([input_scans], axis=0)
    print(qpe_scans.shape)
    return qpe_scans

In [6]:
def data_preprocessing(X):
   # 0. Right shape for batch
   X = X[np.newaxis, ::, ::, ::]
   return X

def data_postprocessing(nwcst):
    # 0. Squeeze empty dimensions
    nwcst = np.squeeze(np.array(nwcst))
    nwcst = nwcst
    nwcst = np.where(nwcst>0, nwcst, 0)
    return nwcst   

def prediction(model_instance, input_data):
    input_data = data_preprocessing(input_data)
    print(input_data.shape)
    pred = model_instance.predict(input_data)
    print(pred.shape)
    nwcst = np.squeeze(pred)  # np.transpose() 대신 np.squeeze()를 사용하세요.
    return nwcst

def save_as_ascii_grid(data, output_file):
    """
    2D NumPy array를 ASCII grid 파일로 저장하는 함수
    :param data: 2D NumPy array (격자 자료)
    :param output_file: 저장할 파일 경로 및 이름 (예: 'output.asc')
    """
    nrows, ncols = data.shape
    header = f"ncols {ncols}\n"
    header += f"nrows {nrows}\n"
    #header += "xllcorner -30934\n"  # x축의 최소값 HFC
    #header += "yllcorner 11140\n"  # y축의 최소값  HFC
    header += "xllcorner -31082\n"  # x축의 최소값 KMA
    header += "yllcorner 11395\n"  # y축의 최소값  KMA
    header += "cellsize 1000.0\n"  # 셀 크기
    header += "NODATA_value -9999.0\n"  # 누락된 데이터를 나타내는 값

    np.savetxt(output_file, data, fmt="%.2f", header=header, comments="", delimiter=" ")


#매선행시점(10분~180분) 최적 예측모델 이용하여 특정시점 예측하기 

In [7]:
from keras.models import load_model
import glob
import numpy as np
import datetime
import os

# 이전에 정의한 data_preprocessing, data_postprocessing, prediction 함수를 사용합니다.

qpf_all_each = np.empty((625, 525, 0))

for j in range(10, 190, 10):
    # 모델 불러오기
    pre_model = f'models/model-best_fcst_{j}min.h5'
    model = load_model(pre_model, custom_objects={"CSI_custom": CSI_custom, "mae_custom": mae_custom})
    
    # 데이터 준비하기
    # FROM NUMPY
    #filename = train_files0[22468]
    #dataset0 = np.load(filename)
    #FROM ASC (KMA mm/hr)
    dataset0=qpe_data('2208082200')
    dataset0.shape
    dataset = np.transpose(dataset0,(1,2,0))
    dataset = np.pad(dataset, [(7, 8), (9, 10), (0, 0)], mode='constant', constant_values=0)
    feature0 = dataset[:, :, :] 
    feature = np.where(feature0 < 0., 0, feature0)

    # 예측하기
    nwcst = prediction(model, feature / 255.)
    nwcst = nwcst * 255.

    qpf_np = nwcst[7:632, 9:534]
    #매 예측시점 자료를 ASCII GRID 형식으로 저장(EPSG518 기준)
    output_file_path = 'QPF/QPF_'+date_nm+'-'+str(j)+'.asc'
    save_as_ascii_grid(qpf_np, output_file_path)
    #이미지 처리를 위한 3차원 배열 예측강우 생성
    qpf_np = np.expand_dims(qpf_np, axis=-1)  # 차원을 늘리세요.
    qpf_all_each = np.concatenate([qpf_all_each, qpf_np], axis=-1)
    

print(qpf_all_each.shape)


2023-08-06 17:01:02.584049: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-06 17:01:03.357205: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 46578 MB memory:  -> device: 0, name: NVIDIA RTX A6000, pci bus id: 0000:65:00.0, compute capability: 8.6
2023-08-06 17:01:03.357893: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1532] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 46711 MB memory:  -> device: 1, name: NVIDIA RTX A6000, pci bus id: 0000:b3:00.0, compute capability: 8.6


2208082130
2208082140
2208082150
2208082200
(4, 625, 525)
(1, 640, 544, 4)


2023-08-06 17:01:05.729650: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8204
2023-08-06 17:01:06.798292: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


1/1 [==============================] - 4s 4s/step
(1, 640, 544, 1)
2208082130
2208082140
2208082150
2208082200
(4, 625, 525)
(1, 640, 544, 4)
1/1 [==============================] - 0s 124ms/step
(1, 640, 544, 1)
2208082130
2208082140
2208082150
2208082200
(4, 625, 525)
(1, 640, 544, 4)
1/1 [==============================] - 0s 120ms/step
(1, 640, 544, 1)
2208082130
2208082140
2208082150
2208082200
(4, 625, 525)
(1, 640, 544, 4)
1/1 [==============================] - 0s 120ms/step
(1, 640, 544, 1)
2208082130
2208082140
2208082150
2208082200
(4, 625, 525)
(1, 640, 544, 4)
1/1 [==============================] - 0s 121ms/step
(1, 640, 544, 1)
2208082130
2208082140
2208082150
2208082200
(4, 625, 525)
(1, 640, 544, 4)
1/1 [==============================] - 0s 123ms/step
(1, 640, 544, 1)
2208082130
2208082140
2208082150
2208082200
(4, 625, 525)
(1, 640, 544, 4)
1/1 [==============================] - 0s 120ms/step
(1, 640, 544, 1)
2208082130
2208082140
2208082150
2208082200
(4, 625, 525)
(1, 6

#재귀적 학습 기반 최적 예측모델 이용하여 특정시점 예측하기 

In [8]:
#개쥐적 학습 기반 강우예측 모델을 이용한 예측강우 생산
from keras.models import load_model
import glob
import numpy as np
import datetime
import os
date_nm='202208082200'
def prediction_rec(model_instance, input_data): #예측시 recursive 안함  
    input_data = data_preprocessing(input_data)
    pred = model_instance.predict(input_data)
    pred=np.squeeze(pred)
    nwcst = np.transpose(pred,(2,0,1))  
    return nwcst

model_rec = load_model('models/model-best_rec_180min_f.h5',custom_objects={"CSI_custom": CSI_custom,"mae_custom":mae_custom}) 

# 데이터 준비하기
#FROM ASC (KMA mm/hr)
dataset0=qpe_data('2208082200')
print('1',dataset0.shape)
dataset = np.transpose(dataset0,(1,2,0))
print('2',dataset.shape)
dataset = np.pad(dataset, [(7, 8), (9, 10), (0, 0)], mode='constant', constant_values=0)
feature0 = dataset[:, :, :] 
feature = np.where(feature0 < 0., 0, feature0)
nwcst = prediction_rec(model_rec, feature/255.)
nwcst = nwcst*255
print('3',nwcst.shape)
#매 예측시점 자료를 ASCII GRID 형식으로 저장(EPSG518 기준)
for j in range(10, 190, 10):
    output_file_path = 'QPF/QPF_REC_'+date_nm+'-'+str(j)+'.asc'
    k=int(j/10-1)
    qpf_np = nwcst[k,7:632, 9:534]
    save_as_ascii_grid(qpf_np, output_file_path)
qpf_all = nwcst[:,7:632, 9:534]
print(qpf_all.shape)


2208082130
2208082140
2208082150
2208082200
(4, 625, 525)
1 (4, 625, 525)
2 (625, 525, 4)
1/1 [==============================] - 3s 3s/step
3 (18, 640, 544)
(18, 625, 525)
